In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
%matplotlib qt

# Part 1
Some constants

In [ ]:
charge = -1.9e-10 # coulombs

initial_vx = 20000 # mm/s
initial_vy = 0 # mm/s
initial_vz = 0 # mm/s
initial_x = 0 # Position, MM
initial_y = 0 # Position, MM
initial_z = 107.95 # Position, MM
initial_t = 0 # Seconds

droplet_diameter = 84e-6 # Meteres
droplet_density = 1000 # kg/m^3
droplet_mass = ( (4/3) * (droplet_diameter/2) ** 3) * math.pi * droplet_density
print_resolution = 300 # dpi
print(droplet_mass)

total_distance_travel = 3 # MM
plate_voltage = 0 # Volts, for part 1
plate_seperation = 0.001 # Meters
plate_start = 1.25 # Position MM
plate_length = 0.5 # MM
plate_end = plate_start + plate_length # Position, MM
paper_end = 3 # MM


## Calculations For Question 1
We are assuming no voltage is applied to the plates. <br>
We are looking for how long it takes for the droplet to reach the paper. <br>
Due to the droplet being so small, we can negate air resistance. Therefore, the velocity will remain constant for X, Y and Z, for this case. <br>
While we have declared a part for the Y and Z component, we will not use it in this case, as there will be zero force applied and zero position changed in the Y and Z direction (for question 1)<br>

In [ ]:
droplet = ({'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]})

electric_field = plate_voltage / plate_seperation # Should be zero in this case, due to no plate voltage

step = 0
time_step = 1e-6

while (droplet['position_x'][step] <= paper_end):
    droplet['time'].append( time_step * (step + 1) )

    if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
        force_z = 0
    else: # If it is between the plates
        force_z = electric_field * charge # This should be zero in this case, due to there being no electric field between the plates

    droplet['velocity_x'].append(initial_vx) # Should not vary here
    droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
    droplet['velocity_y'].append(initial_vy)

    droplet['position_z'].append( (droplet['velocity_z'][step] * time_step) + droplet['position_z'][step] )
    droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
    droplet['position_y'].append(0) # It is known it will not change in this scenario
    
    step = step + 1


## Plotting Question 1

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 1, initial_z + 1, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet Trajectory, when V=0')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(np.min(droplet['position_y']) - 1, np.max(droplet['position_y']) + 1)
ax1.set_xlim(np.min(droplet['position_x']), np.max(droplet['position_x']) + 0.1)
ax1.set_zlim(np.min(droplet['position_z']) - 1, np.max(droplet['position_z']) + 1)
ax1.view_init(elev=5, azim=-105)

dot1 = ax1.scatter([], [], [], color='r', label='Position')

def update_position(frame):
    dot1._offsets3d = (droplet['position_x'][frame:frame+1], droplet['position_y'][frame:frame+1], droplet['position_z'][frame:frame+1])
    return dot1,

ani1 = animation.FuncAnimation(fig1, update_position, frames=step, interval=10, blit=False, repeat=False)

plt.show()

# ani1.save("movie1.mp4", writer='ffmpeg', fps=480)


# Calculating time it took to to reach the paper
print(f"The time it took to reach the paper was {droplet['time'][-1]} seconds")


# Question 2
Simulate the act of drawing a vertical line (similar to the letter “I” ) along the center of the paper at 300 dpi resolution. How much time would it take to draw the letter ‘I’? <br>
In this case, the voltage needs to be dynamically varied accross each step. <br>
This will affect the Z position, which will be calcualted for each dot trajectory <br>
We can approximate that if we are require 300dpi, this equated to about 12 dpmm (dots per milimeter). This tells us how many dots we need to draw per milimeter <br>. 

In [ ]:

plate_voltage = 2330.5 
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    print("Current voltage:",plate_voltage)
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 79


# Plotting Question 2

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 2.75, initial_z + 2.75, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet trajectories when the voltage is varying')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, 3.1)
ax1.set_zlim(initial_z - 2.75, initial_z + 2.75)
ax1.view_init(elev=5, azim=-105)
dots = []
for i, droplet in enumerate(droplets):
    dot = ax1.scatter([], [], [], color='r', label=f'Droplet {i+1}' if i == 0 else "")
    dots.append(dot)

def update_position(frame):
    current_droplet = frame // step 
    current_frame = frame % step
    
    if current_droplet < len(droplets):
        droplet = droplets[current_droplet]
        dots[current_droplet]._offsets3d = (droplet['position_x'][current_frame:current_frame+1],  droplet['position_y'][current_frame:current_frame+1], droplet['position_z'][current_frame:current_frame+1])
    return dots
total_frames = len(droplets) * step
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=False, repeat=False)

plt.show()

# ani1.save("movie1.mp4", writer='ffmpeg', fps=480)
print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1], "seconds")


# Question 3
We are now looking to draw the 'I' as big as possible. This was nearly achieved in the previous section, but it can still be larger. <br>
We are also going to speed up the process, by having it so when a droplet exits the parallel plates, the next droplet will enter. <br>

In [ ]:
plate_voltage = 2360 # This will by dynamically changing to draw an '+'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 80


# Plotting Question 3
The data for plate voltage vs. time was gathered during the calculations done in question 2. The plot for voltage vs. time will be shown below, as a line graph. <br>
It should be seen as a staircase. <br>

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 2.8, initial_z + 2.8, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet trajectories when the voltage is varying')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, 3.1)
ax1.set_zlim(initial_z - 2.75, initial_z + 2.75)
ax1.view_init(elev=5, azim=-105)
dots = []
for i, droplet in enumerate(droplets):
    dot = ax1.scatter([], [], [], color='r', label=f'Droplet {i+1}' if i == 0 else "")
    dots.append(dot)

# Define step based on the length of positions in the first droplet (assuming all are the same)
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

def update_position(frame):
    for i, dot in enumerate(dots):
        sf = start_frames[i]
        if frame < sf:
            dot._offsets3d = ([], [], [])
        elif frame < sf + step:
            lf = frame - sf
            dot._offsets3d = (
                droplets[i]['position_x'][lf:lf+1],
                droplets[i]['position_y'][lf:lf+1],
                droplets[i]['position_z'][lf:lf+1]
            )
        else:
            dot._offsets3d = (
                droplets[i]['position_x'][-1:],
                droplets[i]['position_y'][-1:],
                droplets[i]['position_z'][-1:]
            )
    return dots

ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=False, repeat=False)

plt.show()

# ani1.save("movie1.mp4", writer='ffmpeg', fps=480)
# The time will be halved
print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)

plt.show()